In [4]:
import pandas as pd
import sqlite3

In [2]:
# return SQL query as pandas dataframe
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

In [6]:
# execute SQL command using sqlite module
def run_command(query):
    with sqlite3.connect('chinook.db') as conn:
        # autocommit
        conn.isolation_level = None
        conn.execute(c)

In [7]:
# return tables' name and type in chinook.db
def show_tables():
    query = """
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    """   
    return run_query(query)

In [9]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


In [25]:
track_sold_query = """
WITH usa_sales AS (
    SELECT SUM(il.quantity)
    FROM invoice_line il
    LEFT JOIN invoice i ON il.invoice_id = i.invoice_id
    LEFT JOIN customer c ON i.customer_id = c.customer_id
    WHERE c.country = 'USA'
)

SELECT 
    g.name genre,
    sum(il.quantity) n_sold,
    ROUND(
        CAST(sum(il.quantity) AS FLOAT) /(SELECT * FROM usa_sales)*100
        ,2) n_sold_perc
FROM invoice_line il
LEFT JOIN track t ON il.track_id = t.track_id
LEFT JOIN genre g ON t.genre_id = g.genre_id
LEFT JOIN invoice i ON il.invoice_id = i.invoice_id
LEFT JOIN customer c ON i.customer_id = c.customer_id
WHERE c.country = 'USA'
GROUP BY g.name
ORDER BY 3 DESC
"""

In [26]:
run_query(track_sold_query)

,genre,n_sold,n_sold_perc
0,Rock,561,53.38
1,Alternative & Punk,130,12.37
2,Metal,124,11.80
3,R&B/Soul,53,5.04
4,Blues,36,3.43
5,Alternative,35,3.33
6,Latin,22,2.09
7,Pop,22,2.09
8,Hip Hop/Rap,20,1.90
9,Jazz,14,1.33


In [11]:
test_query = """
 SELECT SUM(quantity)
    FROM invoice_line il
    LEFT JOIN invoice i ON il.invoice_id = i.invoice_id
    WHERE i.billing_country = 'USA'
"""

run_query(test_query)

,SUM(quantity)
0,1051
